works fine on google colab

In [ ]:
import torch
import torch.nn as nn


# import Layer from the utils.py file
from utils import load_tweets, process_tweet   # utils automaticaly data download kr lay ga, data kisi file main pass krnay ki zrurat nahi

import random as rnd

import numpy as np



# Importing the data


## Loading in the data

Import the data set.  
- You may recognize this from earlier assignments in the specialization.
- Details of process_tweet function are available in utils.py file

In [ ]:
## DO NOT EDIT THIS CELL

# Import functions from the utils.py file

import numpy as np

# Load positive and negative tweets
all_positive_tweets, all_negative_tweets = load_tweets()

# View the total number of positive and negative tweets.
print(f"The number of positive tweets: {len(all_positive_tweets)}")
print(f"The number of negative tweets: {len(all_negative_tweets)}")

# Split positive set into validation and training
val_pos   = all_positive_tweets[4000:] # generating validation set for positive tweets
train_pos = all_positive_tweets[:4000]# generating training set for positive tweets

# Split negative set into validation and training
val_neg   = all_negative_tweets[4000:] # generating validation set for negative tweets
train_neg = all_negative_tweets[:4000] # generating training set for nagative tweets

# Combine training data into one set
train_x = train_pos + train_neg

# Combine validation data into one set
val_x  = val_pos + val_neg

# Set the labels for the training set (1 for positive, 0 for negative)
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))

# Set the labels for the validation set (1 for positive, 0 for negative)
val_y  = np.append(np.ones(len(val_pos)), np.zeros(len(val_neg)))

print(f"length of train_x {len(train_x)}")
print(f"length of val_x {len(val_x)}")

The number of positive tweets: 5000
The number of negative tweets: 5000
length of train_x 8000
length of val_x 2000


Now import a function that processes tweets (we've provided this in the utils.py file).
- `process_tweets' removes unwanted characters e.g. hashtag, hyperlinks, stock tickers from tweet.
- It also returns a list of words (it tokenizes the original string).

In [ ]:
# Import a function that processes the tweets
# from utils import process_tweet

# Try out function that processes tweets
print("original tweet at training position 0")
print(train_pos[0])

print("Tweet at training position 0 after processing:")
process_tweet(train_pos[0])

original tweet at training position 0
#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
Tweet at training position 0 after processing:


['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']

## Building the vocabulary

Now build the vocabulary.
- Map each word in each tweet to an integer (an "index").
- The following code does this for you, but please read it and understand what it's doing.
- Note that you will build the vocabulary based on the training data.
- To do so, you will assign an index to everyword by iterating over your training set.

The vocabulary will also include some special tokens
- `__PAD__`: padding
- `</e>`: end of line
- `__UNK__`: a token representing any word that is not in the vocabulary.

In [ ]:
# Build the vocabulary
# Unit Test Note - There is no test set here only train/val

# Include special tokens
# started with pad, end of line and unk tokens
Vocab = {'__PAD__': 0, '__</e>__': 1, '__UNK__': 2}

# Note that we build vocab using training data
for tweet in train_x:
    processed_tweet = process_tweet(tweet)
    for word in processed_tweet:
        if word not in Vocab:
            Vocab[word] = len(Vocab)

print("Total words in vocab are",len(Vocab))
display(Vocab)


Total words in vocab are 9088


{'__PAD__': 0,
 '__</e>__': 1,
 '__UNK__': 2,
 'followfriday': 3,
 'top': 4,
 'engag': 5,
 'member': 6,
 'commun': 7,
 'week': 8,
 ':)': 9,
 'hey': 10,
 'jame': 11,
 'odd': 12,
 ':/': 13,
 'pleas': 14,
 'call': 15,
 'contact': 16,
 'centr': 17,
 '02392441234': 18,
 'abl': 19,
 'assist': 20,
 'mani': 21,
 'thank': 22,
 'listen': 23,
 'last': 24,
 'night': 25,
 'bleed': 26,
 'amaz': 27,
 'track': 28,
 'scotland': 29,
 'congrat': 30,
 'yeaaah': 31,
 'yipppi': 32,
 'accnt': 33,
 'verifi': 34,
 'rqst': 35,
 'succeed': 36,
 'got': 37,
 'blue': 38,
 'tick': 39,
 'mark': 40,
 'fb': 41,
 'profil': 42,
 '15': 43,
 'day': 44,
 'one': 45,
 'irresist': 46,
 'flipkartfashionfriday': 47,
 'like': 48,
 'keep': 49,
 'love': 50,
 'custom': 51,
 'wait': 52,
 'long': 53,
 'hope': 54,
 'enjoy': 55,
 'happi': 56,
 'friday': 57,
 'lwwf': 58,
 'second': 59,
 'thought': 60,
 '’': 61,
 'enough': 62,
 'time': 63,
 'dd': 64,
 'new': 65,
 'short': 66,
 'enter': 67,
 'system': 68,
 'sheep': 69,
 'must': 70,
 'buy':

In [ ]:
len(Vocab)


9088

The dictionary `Vocab` will look like this:
```CPP
{'__PAD__': 0,
 '__</e>__': 1,
 '__UNK__': 2,
 'followfriday': 3,
 'top': 4,
 'engag': 5,
 ...
```

- Each unique word has a unique integer associated with it.
- The total number of words in Vocab: 9088

## Converting a tweet to a tensor

Write a function that will convert each tweet to a tensor (a list of unique integer IDs representing the processed tweet).
- Note, the returned data type will be a **regular Python `list()`**
    - You won't use TensorFlow in this function
    - You also won't use a numpy array
    - You also won't use trax.fastmath.numpy array
- For words in the tweet that are not in the vocabulary, set them to the unique ID for the token `__UNK__`.

##### Example
Input a tweet:
```CPP
'@happypuppy, is Maria happy?'
```

The tweet_to_tensor will first conver the tweet into a list of tokens (including only relevant words)
```CPP
['maria', 'happi']
```

Then it will convert each word into its unique integer

```CPP
[2, 56]
```
- Notice that the word "maria" is not in the vocabulary, so it is assigned the unique integer associated with the `__UNK__` token, because it is considered "unknown."



### Exercise
**Instructions:** Write a program `tweet_to_tensor` that takes in a tweet and converts it to an array of numbers. You can use the `Vocab` dictionary you just found to help create the tensor.

- Use the vocab_dict parameter and not a global variable.
- Do not hard code the integer value for the `__UNK__` token.

In [ ]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: tweet_to_tensor
def tweet_to_tensor(tweet, vocab_dict, unk_token='__UNK__', verbose=False):
    '''
    Input:
        tweet - A string containing a tweet
        vocab_dict - The words dictionary
        unk_token - The special string for unknown tokens
        verbose - Print info durign runtime
    Output:
        tensor_l - A python list with

    '''

    ### START CODE HERE (Replace instances of 'None' with your code) ###
    # Process the tweet into a list of words
    # where only important words are kept (stop words removed)
    word_l = process_tweet(tweet)

    if verbose:
        print("List of words from the processed tweet:")
        print(word_l)

    # Initialize the list that will contain the unique integer IDs of each word
    tensor_l = []              # tensor lsit

    # Get the unique integer ID of the __UNK__ token
    unk_ID = vocab_dict[unk_token]

    if verbose:
        print(f"The unique integer ID for the unk_token is {unk_ID}")

    # for each word in the list:
    for word in word_l:

        # Get the unique integer ID.
        # If the word doesn't exist in the vocab dictionary,
        # use the unique ID for __UNK__ instead.
        word_ID = vocab_dict[word] if word in vocab_dict else unk_ID    # could have also used    vocab_dict.get(word, unk_ID)
    ### END CODE HERE ###

        # Append the unique integer ID to the tensor list.
        tensor_l.append(word_ID)

    return tensor_l

In [ ]:
print("Actual tweet is\n", val_pos[0])         # validation positive
print("\nTensor of tweet:\n", tweet_to_tensor(val_pos[0], vocab_dict=Vocab))          # tweet not tweetS


Actual tweet is
 Bro:U wan cut hair anot,ur hair long Liao bo
Me:since ord liao,take it easy lor treat as save $ leave it longer :)
Bro:LOL Sibei xialan

Tensor of tweet:
 [1064, 136, 478, 2351, 744, 8148, 1122, 744, 53, 2, 2671, 790, 2, 2, 348, 600, 2, 3488, 1016, 596, 4558, 9, 1064, 157, 2, 2]


##### Expected output

```CPP
Actual tweet is
 Bro:U wan cut hair anot,ur hair long Liao bo
Me:since ord liao,take it easy lor treat as save $ leave it longer :)
Bro:LOL Sibei xialan

Tensor of tweet:
 [1065, 136, 479, 2351, 745, 8148, 1123, 745, 53, 2, 2672, 791, 2, 2, 349, 601, 2, 3489, 1017, 597, 4559, 9, 1065, 157, 2, 2]
```

# issue!

note, the word that is 1064 in my vocab, it is 1065 in their vocab.

problem is somewhere b/w 136 and 480. in my vocab there is one less word, and it then changes the unique ids of all the next words, that is the reason (most probably), why the  next test will fail.

In [ ]:
# test tweet_to_tensor

def test_tweet_to_tensor():
    test_cases = [

        {
            "name":"simple_test_check",
            "input": [val_pos[1], Vocab],
            "expected":[444, 2, 304, 567, 56, 9],
            "error":"The function gives bad output for val_pos[1]. Test failed"
        },
        {
            "name":"datatype_check",
            "input":[val_pos[1], Vocab],
            "expected":type([]),
            "error":"Datatype mismatch. Need only list not np.array"
        },
        {
            "name":"without_unk_check",
            "input":[val_pos[1], Vocab],
            "expected":6,
            "error":"Unk word check not done- Please check if you included mapping for unknown word"
        }
    ]
    count = 0
    for test_case in test_cases:

        try:
            if test_case['name'] == "simple_test_check":
                assert test_case["expected"] == tweet_to_tensor(*test_case['input'])
                count += 1
            if test_case['name'] == "datatype_check":
                assert isinstance(tweet_to_tensor(*test_case['input']), test_case["expected"])
                count += 1
            if test_case['name'] == "without_unk_check":
                assert None not in tweet_to_tensor(*test_case['input'])
                count += 1



        except:
            print(test_case['error'])
    if count == 3:
        print("\033[92m All tests passed")
    else:
        print(count," Tests passed out of 3")
test_tweet_to_tensor()

The function gives bad output for val_pos[1]. Test failed
2  Tests passed out of 3


## Creating a batch generator

### However, I won't use this one

Most of the time in Natural Language Processing, and AI in general we use batches when training our data sets.
- If instead of training with batches of examples, you were to train a model with one example at a time, it would take a very long time to train the model.
- You will now build a data generator that takes in the positive/negative tweets and returns a batch of training examples. It returns the model inputs, the targets (positive or negative labels) and the weight for each target (ex: this allows us to can treat some examples as more important to get right than others, but commonly this will all be 1.0).

Once you create the generator, you could include it in a for loop

```CPP
for batch_inputs, batch_targets, batch_example_weights in data_generator:
    ...
```

You can also get a single batch like this:

```CPP
batch_inputs, batch_targets, batch_example_weights = next(data_generator)
```
The generator returns the next batch each time it's called.
- This generator returns the data in a format (tensors) that you could directly use in your model.
- It returns a triple: the inputs, targets, and loss weights:
-- Inputs is a tensor that contains the batch of tweets we put into the model.
-- Targets is the corresponding batch of labels that we train to generate.
-- Loss weights here are just 1s with same shape as targets. Next week, you will use it to mask input padding.

### Exercise
Implement `data_generator`.

In [ ]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED: Data generator
def data_generator(data_pos, data_neg, batch_size, loop, vocab_dict, shuffle=False):
    '''
    Input:
        data_pos - Set of posstive examples
        data_neg - Set of negative examples
        batch_size - number of samples per batch. Must be even
        loop - True or False
        vocab_dict - The words dictionary
        shuffle - Shuffle the data order
    Yield:
        inputs - Subset of positive and negative examples
        targets - The corresponding labels for the subset
        example_weights - An array specifying the importance of each example

    '''
### START GIVEN CODE ###
    # make sure the batch size is an even number
    # to allow an equal number of positive and negative samples
    assert batch_size % 2 == 0

    # Number of positive examples in each batch is half of the batch size
    # same with number of negative examples in each batch
    n_to_take = batch_size // 2

    # Use pos_index to walk through the data_pos array
    # same with neg_index and data_neg
    pos_index = 0
    neg_index = 0

    len_data_pos = len(data_pos)
    len_data_neg = len(data_neg)

    # Get and array with the data indexes
    pos_index_lines = list(range(len_data_pos))
    neg_index_lines = list(range(len_data_neg))

    # shuffle lines if shuffle is set to True
    if shuffle:
        rnd.shuffle(pos_index_lines)
        rnd.shuffle(neg_index_lines)

    stop = False

    # Loop indefinitely
    while not stop:

        # create a batch with positive and negative examples
        batch = []






# First part: Pack n_to_take positive examples
        # Start from pos_index and increment i up to n_to_take
        for i in range(n_to_take):



   # jab aik dfa saari ki saari +ive examples k number k equal number of times, loop chal gai, phir ye block chlay ga
            # If the positive index goes past the positive dataset lenght,
            if pos_index >= len_data_pos:


   # agar function call krtay vaqt kaha tha k, loop = True, tu phir infinitely data generate hota jaye ga
   # otherwise, agar loop = False hay tu sirf aik bar hi +ive examples k number k equal number of times examples hi output aayain gi, randomly (maybe)
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;

                # If user wants to keep re-using the data, reset the index
                pos_index = 0


   # ab choon k data par dobara iterate krnay lga hun so, ab indices reset kr dain gay aik bar, or new config k mutabiq batches generate hoon gay
                if shuffle:
                    # Shuffle the index of the positive sample
                    rnd.shuffle(pos_index_lines)



            # get the tweet as pos_index
            tweet = data_pos[pos_index_lines[pos_index]]

            # convert the tweet into tensors of integers representing the processed words
            tensor = tweet_to_tensor(tweet, vocab_dict)

   # batch main shuru main saari +ive example pri hoon gi maybe shuffled and repeated as well.
            # append the tensor to the batch list
            batch.append(tensor)

            # Increment pos_index by one
            pos_index = pos_index + 1

### END GIVEN CODE ###







### START CODE HERE (Replace instances of 'None' with your code) ###

# Second part: Pack n_to_take negative examples

        # Using the same batch list, start from neg_index and increment i up to n_to_take
        for i in range(n_to_take):

            # If the negative index goes past the negative dataset length,
            if neg_index >= len_data_neg:

                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;

                # If user wants to keep re-using the data, reset the index
                neg_index = 0

                if shuffle:
                    # Shuffle the index of the negative sample
                    rnd.shuffle(neg_index_lines)
            # get the tweet as pos_index
            tweet = data_neg[neg_index_lines[neg_index]]

            # convert the tweet into tensors of integers representing the processed words
            tensor = tweet_to_tensor(tweet, vocab_dict)

            # append the tensor to the batch list
            batch.append(tensor)

            # Increment neg_index by one
            neg_index += 1

### END CODE HERE ###







### START GIVEN CODE ###
        if stop:
            break;

        # Update the start index for positive data
        # so that it's n_to_take positions after the current pos_index
        pos_index += n_to_take

        # Update the start index for negative data
        # so that it's n_to_take positions after the current neg_index
        neg_index += n_to_take

        # Get the max tweet length (the length of the longest tweet)
        # (you will pad all shorter tweets to have this length)
        max_len = max([len(t) for t in batch])                 # ye length tu os tweet ki length k equal hoti jo training set main sb say bari hoti, isay current batch ki sab say lambi tweet ki length k brarbar q rakah?


        # Initialize the input_l, which will
        # store the padded versions of the tensors
        tensor_pad_l = []
        # Pad shorter tweets with zeros
        for tensor in batch:
### END GIVEN CODE ###


### START CODE HERE (Replace instances of 'None' with your code) ###
            # Get the number of positions to pad for this tensor so that it will be max_len long
            n_pad = max_len - len(tensor)

            # Generate a list of zeros, with length n_pad
            pad_l = [0]*n_pad

            # concatenate the tensor and the list of padded zeros
            tensor_pad = tensor + pad_l

            # append the padded tensor to the list of padded tensors
            tensor_pad_l.append(tensor_pad)







        # convert the list of padded tensors to a numpy array
        # and store this as the model inputs
        inputs = np.array(tensor_pad_l)

        # Generate the list of targets for the positive examples (a list of ones)
        # The length is the number of positive examples in the batch
        target_pos = [1]*n_to_take

        # Generate the list of targets for the negative examples (a list of ones)
        # The length is the number of negative examples in the batch
        target_neg = [0]*n_to_take

        # Concatenate the positve and negative targets
        target_l = target_pos + target_neg

        # Convert the target list into a numpy array
        targets = np.array(target_l)

        # Example weights: Treat all examples equally importantly.
        example_weights = np.ones(len(targets))


### END CODE HERE ###







### GIVEN CODE ###
        # note we use yield and not return
        yield inputs, targets, example_weights


Now you can use your data generator to create a data generator for the training data, and another data generator for the validation data.

We will create a third data generator that does not loop, for testing the final accuracy of the model.

In [ ]:
# Set the random number generator for the shuffle procedure
rnd.seed(30)

# Create the training data generator
def train_generator(batch_size, shuffle = False):
    return data_generator(train_pos, train_neg, batch_size, True, Vocab, shuffle)########

# Create the validation data generator
def val_generator(batch_size, shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, True, Vocab, shuffle)##########

# Create the validation data generator
def test_generator(batch_size, shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, False, Vocab, shuffle)

# Get a batch from the train_generator and inspect.
inputs, targets, example_weights = next(train_generator(4, shuffle=True))

# this will print a list of 4 tensors padded with zeros
print(f'Inputs: {inputs}')
print(f'Targets: {targets}')
print(f'Example Weights: {example_weights}')



Inputs: [[2005 4450 3200    9    0    0    0    0    0    0    0]
 [4953  566 2000 1453 5173 3498  141 3498  130  458    9]
 [3760  109  136  582 2929 3968    0    0    0    0    0]
 [ 249 3760    0    0    0    0    0    0    0    0    0]]
Targets: [1 1 0 0]
Example Weights: [1. 1. 1. 1.]


In [ ]:
# Test the train_generator

# Create a data generator for training data,
# which produces batches of size 4 (for tensors and their respective targets)
tmp_data_gen = train_generator(batch_size = 4)

# Call the data generator to get one batch and its targets
tmp_inputs, tmp_targets, tmp_example_weights = next(tmp_data_gen)

print(f"The inputs shape is {tmp_inputs.shape}")
print(f"The targets shape is {tmp_targets.shape}")
print(f"The example weights shape is {tmp_example_weights.shape}")

for i,t in enumerate(tmp_inputs):
    print(f"input tensor: {t}; target {tmp_targets[i]}; example weights {tmp_example_weights[i]}")


The inputs shape is (4, 14)
The targets shape is (4,)
The example weights shape is (4,)
input tensor: [3 4 5 6 7 8 9 0 0 0 0 0 0 0]; target 1; example weights 1.0
input tensor: [10 11 12 13 14 15 16 17 18 19 20  9 21 22]; target 1; example weights 1.0
input tensor: [5736 2900 3760    0    0    0    0    0    0    0    0    0    0    0]; target 0; example weights 1.0
input tensor: [ 857  255 3651 5737  306 4457  566 1229 2766  327 1201 3760    0    0]; target 0; example weights 1.0


##### Expected output

```CPP
The inputs shape is (4, 14)
The targets shape is (4,)
The example weights shape is (4,)
input tensor: [3 4 5 6 7 8 9 0 0 0 0 0 0 0]; target 1; example weights 1
input tensor: [10 11 12 13 14 15 16 17 18 19 20  9 21 22]; target 1; example weights 1
input tensor: [5738 2901 3761    0    0    0    0    0    0    0    0    0    0    0]; target 0; example weights 1
input tensor: [ 858  256 3652 5739  307 4458  567 1230 2767  328 1202 3761    0    0]; target 0; example weights 1
```

Now that you have your train/val generators, you can just call them and they will return tensors which correspond to your tweets in the first column and their corresponding labels in the second column. Now you can go ahead and start building your neural network.

# Creating train and val datasets from my logic

## train dataset

In [ ]:
type(train_pos)

list

In [ ]:
len(train_pos)

4000

In [ ]:
train_pos[0]

'#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)'

In [ ]:
tensor_tweet = tweet_to_tensor(train_pos[1900], Vocab)

In [ ]:
tensor_tweet

[415, 3398, 118]

In [ ]:
train_pos[1900]

'@hayleybrown750 morning Hayley :-)'

In [ ]:
a = 0
for i in range(len(train_pos)):
  current_len = len(tweet_to_tensor(train_pos[i], Vocab))
  if current_len > a:
    a= current_len
a


51

In [ ]:
a = 0
for i in range(len(train_neg)):
  current_len = len(tweet_to_tensor(train_neg[i], Vocab))
  if current_len > a:
    a= current_len
a


28

In [ ]:
all_pos_int_tweets = []

for i in range(len(train_pos)):
  current_tweet = tweet_to_tensor(train_pos[i], Vocab)
  all_pos_int_tweets.append(current_tweet)


In [ ]:
all_pos_int_tweets[0]

[3, 4, 5, 6, 7, 8, 9]

In [ ]:
all_pos_int_tweets_matrix = np.zeros((len(train_pos), 60))
all_pos_int_tweets_matrix.shape

(4000, 60)

In [ ]:
a = [1,2,3,4,5,6,7,8,9,10]
b = np.zeros((2,15))

b[0, : len(a)] = a
b

array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.]])

In [ ]:
for i in range(len(all_pos_int_tweets)):
  all_pos_int_tweets_matrix[i, :len(all_pos_int_tweets[i])] = all_pos_int_tweets[i]


In [ ]:
all_pos_int_tweets_matrix


array([[3.000e+00, 4.000e+00, 5.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+01, 1.100e+01, 1.200e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.300e+01, 2.400e+01, 2.500e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [2.200e+01, 7.910e+02, 7.150e+02, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [4.160e+02, 5.735e+03, 8.010e+02, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.790e+02, 7.300e+01, 9.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

In [ ]:
pos_true_labels = np.ones(len(train_pos))


In [ ]:
pos_true_labels

array([1., 1., 1., ..., 1., 1., 1.])

In [ ]:
pos_true_labels.shape


(4000,)

In [ ]:
all_neg_int_tweets = []

for i in range(len(train_neg)):
  current_tweet = tweet_to_tensor(train_neg[i], Vocab)
  all_neg_int_tweets.append(current_tweet)


In [ ]:
all_neg_int_tweets[0]


[5736, 2900, 3760]

In [ ]:
all_neg_int_tweets_matrix = np.zeros((len(train_neg), 60))
all_neg_int_tweets_matrix.shape


(4000, 60)

In [ ]:
for i in range(len(all_neg_int_tweets)):
  all_neg_int_tweets_matrix[i, :len(all_neg_int_tweets[i])] = all_neg_int_tweets[i]


In [ ]:
all_neg_int_tweets_matrix


array([[5736., 2900., 3760., ...,    0.,    0.,    0.],
       [ 857.,  255., 3651., ...,    0.,    0.,    0.],
       [1035., 5738., 1428., ...,    0.,    0.,    0.],
       ...,
       [ 363.,  308.,  976., ...,    0.,    0.,    0.],
       [ 134., 1594.,  222., ...,    0.,    0.,    0.],
       [  37., 1308.,  117., ...,    0.,    0.,    0.]])

In [ ]:
all_neg_int_tweets_matrix.shape

(4000, 60)

In [ ]:
neg_true_labels = np.zeros(len(train_neg))


In [ ]:
neg_true_labels

array([0., 0., 0., ..., 0., 0., 0.])

## test dataset

In [ ]:
type(train_x)

list

In [ ]:
len(train_x)


8000

In [ ]:
def list_of_sentences_to_numpy_sequences(list_of_sentences, vocab, labels_of_sentences):

  len_of_list_of_sentences = len(list_of_sentences)

  # extracting the length of largest sequence
  max_len = 0
  for i in range(len_of_list_of_sentences):
    current_len = len(tweet_to_tensor(list_of_sentences[i], vocab))
    if current_len > max_len:
      max_len = current_len


  #
  all_int_sequences = []
  for i in range(len_of_list_of_sentences):
    current_tweet = tweet_to_tensor(list_of_sentences[i], vocab)
    all_int_sequences.append(current_tweet)


  all_int_sequences_matrix = np.zeros((len(list_of_sentences), max_len))

  for i in range(len_of_list_of_sentences):
    all_int_sequences_matrix[i, :len(all_int_sequences[i])] = all_int_sequences[i]

    # labels_of_sentences = int(label_of_sentences)

    # labels = np.full(len_of_list_of_sentences, fill_value=label_of_sentences)                      # np.full(shape, fill_value, dtype=None, order='C')

#  return max_len, all_int_sequences, all_int_sequences_matrix, labels_of_sentences
  return all_int_sequences_matrix, labels_of_sentences


In [ ]:
# a, b, c, d = list_of_sentences_to_numpy_sequences(train_x, Vocab, train_y)

# a, len(b), c, d


(51,
 8000,
 array([[   3.,    4.,    5., ...,    0.,    0.,    0.],
        [  10.,   11.,   12., ...,    0.,    0.,    0.],
        [  23.,   24.,   25., ...,    0.,    0.,    0.],
        ...,
        [ 363.,  308.,  976., ...,    0.,    0.,    0.],
        [ 134., 1594.,  222., ...,    0.,    0.,    0.],
        [  37., 1308.,  117., ...,    0.,    0.,    0.]]),
 array([1., 1., 1., ..., 0., 0., 0.]))

In [ ]:
# c[0,:] == all_pos_int_tweets_matrix[0,:51]


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [ ]:
# c[1000,:] == all_pos_int_tweets_matrix[1000,:51]
# #perfect


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [ ]:
# d[:4001]


array([1., 1., 1., ..., 1., 1., 0.])

In [ ]:
# train_y[:4001]


array([1., 1., 1., ..., 1., 1., 0.])

In [ ]:
# d.shape


(8000,)

In [ ]:
# neg_true_labels
# pos_true_labels

# all_neg_int_tweets_matrix
# all_pos_int_tweets_matrix

In [ ]:
train_X, train_Y = list_of_sentences_to_numpy_sequences(train_x, Vocab, train_y)
# train_neg_int_tweets_matrix, train_neg_labels = list_of_sentences_to_numpy_sequences(train_neg, Vocab, 0)

val_X, val_Y = list_of_sentences_to_numpy_sequences(val_x, Vocab, val_y)
# val_neg_int_tweets_matrix, val_neg_labels = list_of_sentences_to_numpy_sequences(val_neg, Vocab, 0)

# test data bnaya hi nahi, bas train and val hay
# test_pos_int_tweets_matrix, test_pos_labels = list_of_sentences_to_numpy_sequences(test_pos, Vocab, 1)
# test_neg_int_tweets_matrix, test_neg_labels = list_of_sentences_to_numpy_sequences(test_neg, Vocab, 0)


In [ ]:
len(train_Y), train_X.shape


(8000, (8000, 51))

In [ ]:
len(val_X), val_X.shape


(2000, (2000, 31))

In [ ]:
# len(val_pos_labels), val_pos_int_tweets_matrix.shape


(1000, (1000, 31))

In [ ]:
# len(val_neg_labels), val_neg_int_tweets_matrix.shape


(1000, (1000, 24))

# Creating custom layers

In [ ]:
# import torch
# import torch.nn as nn

# Define the custom ReLU class
class Relu(nn.Module):
    """Relu activation function implementation"""

    def forward(self, x):
        '''
        Input:
            - x (a torch.Tensor): the input tensor
        Output:
            - activation (torch.Tensor): all positive or 0 version of x
        '''
        ### START CODE HERE (Replace instances of 'None' with your code) ###

        activation = torch.maximum(x, torch.tensor(0.0))

        ### END CODE HERE ###

        return activation


### Explanation
1. **Custom ReLU Class**: Define a class `Relu` that inherits from `nn.Module`. This class implements the ReLU activation function.
2. **Forward Method**: The `forward` method takes a tensor `x` as input and applies the ReLU function using `torch.maximum`.
3. **Testing**: Create a sample tensor `x`, instantiate the `Relu` class, and print the output after applying the custom ReLU function.

In [ ]:
# Test your relu function

x = np.array([[-2.0, -1.0, 0.0], [0.0, 1.0, 2.0]], dtype=float)
x = torch.from_numpy(x)

relu_layer = Relu()
print("Test data is:")
print(x)
print("Output of Relu is:")
print(relu_layer(x))


Test data is:
tensor([[-2., -1.,  0.],
        [ 0.,  1.,  2.]], dtype=torch.float64)
Output of Relu is:
tensor([[0., 0., 0.],
        [0., 1., 2.]], dtype=torch.float64)


In [ ]:
#import torch

print("Choose a matrix with 2 rows and 3 columns")
tmp_shape = (2, 3)
print(tmp_shape)

# Generate a specific random generator
generator = torch.Generator()
generator.manual_seed(1)  # Seed the generator

# Generate a weight matrix with a normal distribution (mean=0, std=1) using the generator
tmp_weight = torch.randn(tmp_shape, generator=generator)

print("Weight matrix generated with a normal distribution with mean 0 and stdev of 1")
print(tmp_weight)


Choose a matrix with 2 rows and 3 columns
(2, 3)
Weight matrix generated with a normal distribution with mean 0 and stdev of 1
tensor([[ 0.6614,  0.2669,  0.0617],
        [ 0.6213, -0.4519, -0.1661]])


In [ ]:
# cell 2
generator.manual_seed(1)  # Reset the seed
# This ensures that the random numbers generated are the
# same every time you reset the seed of the generator before generating the tensor.


tmp_weight_2 = torch.randn(tmp_shape, generator=generator)
tmp_weight_2


tensor([[ 0.6614,  0.2669,  0.0617],
        [ 0.6213, -0.4519, -0.1661]])

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F


Define the Dense class:

In [ ]:
# import torch
# import torch.nn as nn

class Dense(nn.Module):
    """
    A dense (fully-connected) layer.
    """
    def __init__(self, n_units, init_stdev=0.1):
        super(Dense, self).__init__()

        # Set the number of units in this layer
        self._n_units    = n_units
        self._init_stdev = init_stdev
#        self.weights = None

    def forward(self, x):
        # Matrix multiply x and the weight matrix
        return torch.matmul(x, self.weights)

    def init_weights_and_state(self, input_signature, generator):
        # The input_signature has a .shape attribute that gives the shape as a tuple
        input_shape = input_signature.shape

        # Generate the weight matrix from a normal distribution, with standard deviation of 'stdev'
        w = self._init_stdev * torch.randn((input_shape[-1], self._n_units), generator=generator)

        self.weights = w                   #nn.Parameter(w)  # Convert weights to a parameter so that they are trainable

        return self.weights


### from chat gpt

In Python, when you define a class that inherits from another class, it’s important to call the parent class’s initialization method. This is done using super(). In the context of PyTorch's nn.Module, here’s why calling super(Dense, self).__init__() is necessary:
Why Call super() in PyTorch Modules

    Initialization of Parent Class (nn.Module):
        Base Functionality: nn.Module provides essential functionality for all neural network layers in PyTorch, including parameter management, device handling, and state dict management.
        Internal Mechanisms: The __init__ method of nn.Module initializes internal data structures and mechanisms needed for PyTorch to properly manage and train the network.

    Registering Submodules and Parameters:
        Parameter Registration: By calling super().__init__(), the parent class's initialization code ensures that any parameters or submodules you define in your class are properly registered with PyTorch’s internal mechanisms. This is crucial for the parameters to be included in the model’s list of parameters for optimization and gradient computation.
        Hooks and Other Features: PyTorch’s nn.Module also sets up hooks and other features needed for model training and evaluation. Calling super().__init__() ensures these features are properly initialized.

    Consistency and Best Practices:
        Consistent Behavior: By calling super().__init__(), you ensure that your custom module behaves consistently with other PyTorch modules. This helps avoid issues that might arise from incomplete initialization.
        Avoiding Bugs: Failing to call super().__init__() can lead to bugs and unexpected behavior, as the base class’s initialization code might not run, leaving the object in an incomplete state.

 Testing Dense Class

In [ ]:
# Define the Dense layer with a specified number of units
dense_layer = Dense(n_units=10)  # Sets number of units in dense layer

generator.manual_seed(1)
# Define the input tensor
z = torch.tensor([[2.0, 7.0, 25.0]])  # Input array

# Initialize weights
dense_layer.init_weights_and_state(z, generator)

# Print the weights
print("Weights are\n", dense_layer.weights)

# Forward pass
output = dense_layer(z)

# Print the forward function output
print("Forward function output is", output)


Weights are
 tensor([[-0.1526, -0.0750, -0.0654, -0.1609, -0.0100, -0.0609, -0.0980, -0.1609,
         -0.0712,  0.0304],
        [-0.0777, -0.0251, -0.0222,  0.1687, -0.0321, -0.0299,  0.1879, -0.0072,
          0.0158, -0.0773],
        [ 0.0199,  0.0046, -0.1392,  0.2689, -0.0111,  0.0293, -0.0158, -0.0029,
          0.2357, -0.1037]])
Forward function output is tensor([[-0.3516, -0.2118, -3.7673,  7.5818, -0.5220,  0.4005,  0.7250, -0.4443,
          5.8608, -3.0740]])


In [ ]:
# import torch
# import torch.nn as nn

# Define a Linear layer
linear_layer = nn.Linear(in_features=3, out_features=10)

# Example input tensor
input_tensor = torch.tensor([[2.0, 7.0, 25.0], [1,2,3]])

# Get the output tensor
output_tensor = linear_layer(input_tensor)

print("Output tensor:\n", output_tensor)


Output tensor:
 tensor([[-11.9407,  -9.6725,  13.0987, -10.7307, -15.5369,   2.3867,  -0.9074,
         -14.1406,  -7.2472,   9.6960],
        [ -1.6553,  -2.2163,   1.4720,  -1.5027,  -2.3124,   0.6033,  -0.3181,
          -1.9201,  -0.9589,   0.9671]], grad_fn=<AddmmBackward0>)


### Testing against assignment


In [ ]:
# dense_layer.weights.shape

In [ ]:
# # import torch
# # import torch.nn as nn

# class Dense(nn.Module):
#     """
#     A dense (fully-connected) layer.
#     """
#     def __init__(self, n_units, init_stdev=0.1):
#         super(Dense, self).__init__()

#         # Set the number of units in this layer
#         self._n_units = n_units
#         self._init_stdev = init_stdev
# #        self.weights = None

#     def forward(self, x):
#         # Matrix multiply x and the weight matrix
#         return torch.matmul(x, self.weights)

#     def init_weights_and_state(self, w):
#         self.weights = w

#         return self.weights


In [ ]:
w = torch.tensor([[-0.02837107,  0.09368163, -0.10050073,  0.14165013,  0.10543301,0.09108127, -0.04265671,  0.0986188 , -0.05575324,  0.0015325 ],
       [-0.2078568 ,  0.05548371,  0.09142365,  0.05744596,  0.07227863, 0.01210618, -0.03237354,  0.16234998,  0.02450039, -0.13809781],
       [-0.06111237,  0.01403725,  0.08410043, -0.10943579, -0.1077502 , -0.11396457, -0.0593338 , -0.01557651, -0.03832145, -0.11144515]])

In [ ]:
# w.shape

In [ ]:
# # Define the Dense layer with a specified number of units
# dense_layer = Dense(n_units=10)  # Sets number of units in dense layer

# # Initialize weights
# dense_layer.init_weights_and_state(w)

# # Print the weights
# print("Weights are\n", dense_layer.weights)

# # Forward pass
# output = dense_layer(z)

# # Print the forward function output
# print("Forward function output is", output)


## Model

Now you will implement a classifier using neural networks. Here is the model architecture you will be implementing.

<img src = "nn.jpg" style="width:400px;height:250px;"/>



Please use the `help` function to view documentation for each layer.

In [ ]:
# View documentation on nn.Linear
help(nn.Linear)


Help on class Linear in module torch.nn.modules.linear:

class Linear(torch.nn.modules.module.Module)
 |  Linear(in_features: int, out_features: int, bias: bool = True, device=None, dtype=None) -> None
 |  
 |  Applies a linear transformation to the incoming data: :math:`y = xA^T + b`.
 |  
 |  This module supports :ref:`TensorFloat32<tf32_on_ampere>`.
 |  
 |  On certain ROCm devices, when using float16 inputs this module will use :ref:`different precision<fp16_on_mi200>` for backward.
 |  
 |  Args:
 |      in_features: size of each input sample
 |      out_features: size of each output sample
 |      bias: If set to ``False``, the layer will not learn an additive bias.
 |          Default: ``True``
 |  
 |  Shape:
 |      - Input: :math:`(*, H_{in})` where :math:`*` means any number of
 |        dimensions including none and :math:`H_{in} = \text{in\_features}`.
 |      - Output: :math:`(*, H_{out})` where all but the last dimension
 |        are the same shape as the input and :mat

In [ ]:

# View documentation on nn.Embedding
help(nn.Embedding)


Help on class Embedding in module torch.nn.modules.sparse:

class Embedding(torch.nn.modules.module.Module)
 |  Embedding(num_embeddings: int, embedding_dim: int, padding_idx: Optional[int] = None, max_norm: Optional[float] = None, norm_type: float = 2.0, scale_grad_by_freq: bool = False, sparse: bool = False, _weight: Optional[torch.Tensor] = None, _freeze: bool = False, device=None, dtype=None) -> None
 |  
 |  A simple lookup table that stores embeddings of a fixed dictionary and size.
 |  
 |  This module is often used to store word embeddings and retrieve them using indices.
 |  The input to the module is a list of indices, and the output is the corresponding
 |  word embeddings.
 |  
 |  Args:
 |      num_embeddings (int): size of the dictionary of embeddings
 |      embedding_dim (int): the size of each embedding vector
 |      padding_idx (int, optional): If specified, the entries at :attr:`padding_idx` do not contribute to the gradient;
 |                                   the

In [ ]:
# import torch
# import torch.nn as nn

# Define an Embedding layer
embedding = nn.Embedding(num_embeddings=10, embedding_dim=3)        # num_embeddings: maximum possible unique id integer of a word possible to input

# Example input (indices)
input_indices = torch.tensor([[1, 2, 3, 4, 5, 6 ,7 ,8,9,9,9,9,9,9], [1, 2, 3, 4, 5, 6 ,7 ,8,9,9,9,9,9,9]])

# Get embeddings for the input indices
output_embeddings = embedding(input_indices)

print("Output embeddings:\n", output_embeddings)


Output embeddings:
 tensor([[[ 0.1366, -0.4204, -0.4233],
         [-0.2332,  0.7853, -3.5290],
         [ 0.5739, -1.1590,  0.8263],
         [ 1.1203,  1.0225,  0.4271],
         [-1.9306, -0.3250, -0.4264],
         [-0.4470, -0.6684, -0.7939],
         [-0.0971, -0.6128,  2.5988],
         [-1.2032,  1.9522, -0.2459],
         [ 1.1688,  1.3003,  0.2577],
         [ 1.1688,  1.3003,  0.2577],
         [ 1.1688,  1.3003,  0.2577],
         [ 1.1688,  1.3003,  0.2577],
         [ 1.1688,  1.3003,  0.2577],
         [ 1.1688,  1.3003,  0.2577]],

        [[ 0.1366, -0.4204, -0.4233],
         [-0.2332,  0.7853, -3.5290],
         [ 0.5739, -1.1590,  0.8263],
         [ 1.1203,  1.0225,  0.4271],
         [-1.9306, -0.3250, -0.4264],
         [-0.4470, -0.6684, -0.7939],
         [-0.0971, -0.6128,  2.5988],
         [-1.2032,  1.9522, -0.2459],
         [ 1.1688,  1.3003,  0.2577],
         [ 1.1688,  1.3003,  0.2577],
         [ 1.1688,  1.3003,  0.2577],
         [ 1.1688,  1.3003, 

In [ ]:
# # erro expected

# # import torch
# # import torch.nn as nn

# # Define an Embedding layer
# embedding = nn.Embedding(num_embeddings=10, embedding_dim=3)

# # Example input (indices)
# input_indices = torch.tensor([1,10])    # 10 was not possible!

# # Get embeddings for the input indices
# output_embeddings = embedding(input_indices)

# print("Output embeddings:\n", output_embeddings)


IndexError: index out of range in self

- [tl.Mean](https://github.com/google/trax/blob/1372b903bb66b0daccee19fd0b1fdf44f659330b/trax/layers/core.py#L276): Calculates means across an axis.  In this case, please choose axis = 1 to get an average embedding vector (an embedding vector that is an average of all words in the vocabulary).  
- For example, if the embedding matrix is 300 elements and vocab size is 10,000 words, taking the mean of the embedding matrix along axis=1 will yield a vector of 300 elements.

In [ ]:
# # error expected, pytorch main mean layer nahi hay

# # View documentation on nn.Embedding
# help(nn.mean)



In [ ]:
# import torch

# Create a sample tensor
tensor = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
print(tensor)

# Compute the mean of the tensor along a specified dimension
mean_all  = torch.mean(tensor)                # Mean of all elements
mean_dim0 = torch.mean(tensor, dim=0)        # Mean along the rows (dimension 0)
mean_dim1 = torch.mean(tensor, dim=1)        # Mean along the columns (dimension 1)


print("Mean of all elements:\n", mean_all)
print("Mean along dimension 0:\n", mean_dim0)
print("Mean along dimension 1:\n", mean_dim1)


tensor([[1., 2., 3.],
        [4., 5., 6.]])
Mean of all elements:
 tensor(3.5000)
Mean along dimension 0:
 tensor([2.5000, 3.5000, 4.5000])
Mean along dimension 1:
 tensor([2., 5.])


In [ ]:
tensor.shape


torch.Size([2, 3])

In [ ]:
tensor.mean(dim=0)


tensor([2.5000, 3.5000, 4.5000])

In [ ]:
help(nn.LogSoftmax(dim=1))


Help on LogSoftmax in module torch.nn.modules.activation object:

class LogSoftmax(torch.nn.modules.module.Module)
 |  LogSoftmax(dim: Optional[int] = None) -> None
 |  
 |  Applies the :math:`\log(\text{Softmax}(x))` function to an n-dimensional input Tensor.
 |  
 |  The LogSoftmax formulation can be simplified as:
 |  
 |  .. math::
 |      \text{LogSoftmax}(x_{i}) = \log\left(\frac{\exp(x_i) }{ \sum_j \exp(x_j)} \right)
 |  
 |  Shape:
 |      - Input: :math:`(*)` where `*` means, any number of additional
 |        dimensions
 |      - Output: :math:`(*)`, same shape as the input
 |  
 |  Args:
 |      dim (int): A dimension along which LogSoftmax will be computed.
 |  
 |  Returns:
 |      a Tensor of the same dimension and shape as the input with
 |      values in the range [-inf, 0)
 |  
 |  Examples::
 |  
 |      >>> m = nn.LogSoftmax(dim=1)
 |      >>> input = torch.randn(2, 3)
 |      >>> output = m(input)
 |  
 |  Method resolution order:
 |      LogSoftmax
 |      torch.nn

# Creating the Model


### Creating generators

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Combine positive and negative samples
all_tweets_matrix = np.concatenate((all_pos_int_tweets_matrix, all_neg_int_tweets_matrix), axis=0)
all_labels = np.concatenate((pos_true_labels, neg_true_labels), axis=0)

# Convert to PyTorch tensors
tweets_tensor = torch.tensor(all_tweets_matrix, dtype=torch.long)
labels_tensor = torch.tensor(all_labels, dtype=torch.long)

# Create a dataset and dataloader
dataset = TensorDataset(tweets_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


NameError: name 'all_pos_int_tweets_matrix' is not defined

In [ ]:
all_tweets_matrix.shape

(8000, 60)

In [ ]:
len(list(dataset))

8000

In [ ]:
dataset

In [ ]:
list(dataset)[7000]

(tensor([8339, 3760,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]),
 tensor(0.))

In [ ]:
list(dataset)[1000]

(tensor([   7, 2100,    9,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]),
 tensor(1))

In [ ]:
type(list(dataset)[1000])

tuple

In [ ]:
dataloader


In [ ]:
list(dataloader)

[[tensor([[7570,  116, 3760,  ...,    0,    0,    0],
          [ 387,  388,  389,  ...,    0,    0,    0],
          [ 498, 1392, 1673,  ...,    0,    0,    0],
          ...,
          [ 246,  551, 3510,  ...,    0,    0,    0],
          [ 130, 3760,    0,  ...,    0,    0,    0],
          [ 566,  838,  933,  ...,    0,    0,    0]]),
  tensor([0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
          0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
          0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0.,
          0., 0., 0., 1., 1., 1., 0., 0., 0., 0.])],
 [tensor([[ 705,   60, 3227,  ...,    0,    0,    0],
          [2724,  566,  352,  ...,    0,    0,    0],
          [2597, 8839, 3760,  ...,    0,    0,    0],
          ...,
          [6066, 1613, 1329,  ...,    0,    0,    0],
          [ 458,  873, 5679,  ...,    0,    0,    0],
          [ 599,  319, 1868,  ...,    0,    0,    0]]),
  tensor([1., 0., 0

In [ ]:
len(list(dataloader)) # 250 batch pray huye hain


250

In [ ]:
list(dataloader)[0]  # first batch

NameError: name 'dataloader' is not defined

In [ ]:
type(list(dataloader)[0])

NameError: name 'dataloader' is not defined

In [ ]:
len(list(dataloader)[0])  # 0: tensor tweets, 1: corresponding labels 0s and 1s


2

In [ ]:
list(dataloader)[0][0]


tensor([[1654,  236, 1109,  ...,    0,    0,    0],
        [5747,  134, 7598,  ...,    0,    0,    0],
        [  22,    9,    0,  ...,    0,    0,    0],
        ...,
        [ 376, 8313, 8313,  ...,    0,    0,    0],
        [ 311,  330, 5278,  ...,    0,    0,    0],
        [  37, 1706,  420,  ...,    0,    0,    0]])

In [ ]:
len(list(dataloader)[0][0])


64

In [ ]:
list(dataloader)[0][1]

tensor([0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1.,
        1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
        1., 1., 0., 0., 1., 1., 1., 0., 1., 0.])

In [ ]:
list(dataloader)[0][1] == 0

tensor([False,  True, False, False,  True, False,  True, False,  True, False,
         True, False, False,  True,  True, False, False,  True, False, False,
         True,  True,  True,  True, False, False, False, False, False, False,
        False, False,  True, False,  True, False, False,  True, False,  True,
        False, False, False,  True, False,  True,  True, False, False, False,
        False, False, False, False, False,  True,  True, False,  True,  True,
        False,  True,  True, False])

In [ ]:
torch.sum(list(dataloader)[0][1] == 0)
# yani 64 main say 30 0s hain

tensor(30)

### Model Architecture
#### Raw logits are model's outputs

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class TweetClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim):
        super(TweetClassifier, self).__init__()

        self.embedding  =  nn.Embedding(vocab_size, embedding_dim)
        self.fc         =  nn.Linear(embedding_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        embedded = embedded.mean(dim=1)
        output = self.fc(embedded)
        return output


In [ ]:
# Example vocab size (you need to replace this with the actual size of your vocab)
vocab_size    = len(Vocab)
embedding_dim = 256
output_dim    = 1

model = TweetClassifier(vocab_size, embedding_dim, output_dim)


In [ ]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()                           # binary cross entropy with logits
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5




Epoch [1/5], Loss: 0.0880
Epoch [2/5], Loss: 0.0332
Epoch [3/5], Loss: 0.0546
Epoch [4/5], Loss: 0.0151
Epoch [5/5], Loss: 0.0258


### Training

In [ ]:
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in dataloader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
sample = torch.from_numpy(all_pos_int_tweets_matrix[0,:]).long()
sample

tensor([3, 4, 5, 6, 7, 8, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
sample.shape

torch.Size([60])

In [ ]:
sample = sample.unsqueeze(0)
sample.shape

torch.Size([1, 60])

In [ ]:
model.eval()

TweetClassifier(
  (embedding): Embedding(9088, 256)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [ ]:
9088*256

2326528

In [ ]:
with torch.no_grad():
    outputs = model(sample)
    predictions = torch.sigmoid(outputs).squeeze()

predictions


tensor(0.9993)

In [ ]:
sentiment = torch.sigmoid(model(sample))
sentiment


tensor([[0.9993]], grad_fn=<SigmoidBackward0>)

In [ ]:
model(sample)


tensor([[7.2864]], grad_fn=<AddmmBackward0>)

# Variation of Model

In [ ]:
# Combine positive and negative samples
all_tweets_matrix = np.vstack((all_pos_int_tweets_matrix, all_neg_int_tweets_matrix))
all_labels = np.concatenate((pos_true_labels, neg_true_labels))

# Convert to PyTorch tensors
tweets_tensor = torch.tensor(all_tweets_matrix, dtype=torch.long)
labels_tensor = torch.tensor(all_labels, dtype=torch.float32)

# Create a TensorDataset and DataLoader
dataset = TensorDataset(tweets_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


### Model Architecture
#### Sigmoid is model's output

In [ ]:
# Step 2: Define the neural network architecture
class SentimentAnalysisModel(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(SentimentAnalysisModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc = nn.Linear(embed_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        embedded = embedded.mean(dim=1)  # Average embeddings over the sequence length
        output = self.fc(embedded)
        output = self.sigmoid(output)
        return output


In [ ]:
# Parameters
vocab_size = 9088
embed_dim = 256

# Instantiate the model
model = SentimentAnalysisModel(vocab_size, embed_dim)

# Step 3: Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Step 4: Train the model
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

Epoch 1/10, Loss: 0.6813698740005493
Epoch 2/10, Loss: 0.6301184883117675
Epoch 3/10, Loss: 0.557199560880661
Epoch 4/10, Loss: 0.46323020124435427
Epoch 5/10, Loss: 0.36554393291473386
Epoch 6/10, Loss: 0.28155439710617064
Epoch 7/10, Loss: 0.21524840998649597
Epoch 8/10, Loss: 0.16630184495449066
Epoch 9/10, Loss: 0.1301447246670723
Epoch 10/10, Loss: 0.10356896281242371


# Another Model Variation

### with softmax output and neg-log-likelihood loss fn

In [ ]:
# Combine positive and negative samples
all_tweets_matrix = np.vstack((all_pos_int_tweets_matrix, all_neg_int_tweets_matrix))
all_labels = np.concatenate((pos_true_labels, neg_true_labels))

# Convert to PyTorch tensors
tweets_tensor = torch.tensor(all_tweets_matrix, dtype=torch.long)
labels_tensor = torch.tensor(all_labels, dtype=torch.long)  # Change to long for CrossEntropyLoss

# Create a TensorDataset and DataLoader
dataset = TensorDataset(tweets_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


In [ ]:
# Step 2: Define the neural network architecture
class SentimentAnalysisModel(nn.Module):
  def __init__(self, vocab_size, embed_dim):

    super(SentimentAnalysisModel, self).__init__()
    self.embedding   =  nn.Embedding(vocab_size, embed_dim)
    self.fc          =  nn.Linear(embed_dim, 2)                 # Output dimension is 2 for softmax
    self.log_softmax =  nn.LogSoftmax(dim=1)

  def forward(self, x):
    embedded      =  self.embedding(x)
    embedded      =  embedded.mean(dim=1)                      # Average embeddings over the sequence length
    logits        =  self.fc(embedded)
    probabilities =  self.log_softmax(logits)                      # Apply softmax to get probabilities
    return probabilities



In [ ]:
# Parameters
vocab_size = 9088
embed_dim  = 256

# Instantiate the model
model = SentimentAnalysisModel(vocab_size, embed_dim)


In [ ]:
# Step 3: Define loss function and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Step 4: Train the model
num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        probabilities = model(inputs)  # Get probabilities
        loss = criterion(probabilities, labels)  # CrossEntropyLoss expects class indices
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")



Epoch 1/30, Loss: 0.6684565997123718
Epoch 2/30, Loss: 0.5714530947208405
Epoch 3/30, Loss: 0.44834967255592345
Epoch 4/30, Loss: 0.32567218828201294
Epoch 5/30, Loss: 0.23065419149398803
Epoch 6/30, Loss: 0.1652928866147995
Epoch 7/30, Loss: 0.12165165191888809
Epoch 8/30, Loss: 0.09211435216665267
Epoch 9/30, Loss: 0.07155854806303978
Epoch 10/30, Loss: 0.057603250116109846
Epoch 11/30, Loss: 0.04688794532418251
Epoch 12/30, Loss: 0.039139284729957584
Epoch 13/30, Loss: 0.03312096880376339
Epoch 14/30, Loss: 0.02849630557745695
Epoch 15/30, Loss: 0.02510268522799015
Epoch 16/30, Loss: 0.02176929384469986
Epoch 17/30, Loss: 0.01927870847284794
Epoch 18/30, Loss: 0.01721077525615692
Epoch 19/30, Loss: 0.015545440148562193
Epoch 20/30, Loss: 0.013869173765182496
Epoch 21/30, Loss: 0.012771548934280872
Epoch 22/30, Loss: 0.01172525018081069
Epoch 23/30, Loss: 0.010839160164818168
Epoch 24/30, Loss: 0.009842290868982672
Epoch 25/30, Loss: 0.009278351612389088
Epoch 26/30, Loss: 0.00854662

# Another model Variation

### raw logits output, crossentropy loss

nn.CrossEntropyLoss combines nn.LogSoftmax() and nn.NLLLoss() (negative log likelihood loss) in a single class.

It expects the raw logits (unnormalized scores) as input, not probabilities.

The softmax function within nn.CrossEntropyLoss will handle the conversion to probabilities internally.

In [ ]:
# Combine positive and negative samples
all_tweets_matrix = np.vstack((all_pos_int_tweets_matrix, all_neg_int_tweets_matrix))
all_labels = np.concatenate((pos_true_labels, neg_true_labels))

# Convert to PyTorch tensors
tweets_tensor = torch.tensor(all_tweets_matrix, dtype=torch.long)
labels_tensor = torch.tensor(all_labels, dtype=torch.long)  # Change to long for CrossEntropyLoss

# Create a TensorDataset and DataLoader
dataset = TensorDataset(tweets_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


In [ ]:
# Step 2: Define the neural network architecture
class SentimentAnalysisModel(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(SentimentAnalysisModel, self).__init__()

        self.embedding  =  nn.Embedding(vocab_size, embed_dim)
        self.fc         =  nn.Linear(embed_dim, 2)  # Output dimension is 2 for softmax

        # self.softmax = nn.Softmax(dim=1)   # Softmax over the classes


    def forward(self, x):
        embedded  =  self.embedding(x)
        embedded  =  embedded.mean(dim=1)  # Average embeddings over the sequence length
        logits    =  self.fc(embedded)
        return logits  # Return logits for CrossEntropyLoss



In [ ]:
# Parameters
vocab_size = len(Vocab)
embed_dim = 256

# Instantiate the model
model = SentimentAnalysisModel(vocab_size, embed_dim)


In [ ]:
# Step 3: Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Step 4: Train the model
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        optimizer.zero_grad()

        logits = model(inputs)

        loss   = criterion(logits, labels)  # CrossEntropyLoss expects class indices

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")
    print('logits: ', logits.shape, 'label: ', labels.shape)
    print()



Epoch 1/10, Loss: 0.6698989157676697
logits:  torch.Size([64, 2]) label:  torch.Size([64])

Epoch 2/10, Loss: 0.5840957436561585
logits:  torch.Size([64, 2]) label:  torch.Size([64])

Epoch 3/10, Loss: 0.4701932153701782
logits:  torch.Size([64, 2]) label:  torch.Size([64])

Epoch 4/10, Loss: 0.348238805770874
logits:  torch.Size([64, 2]) label:  torch.Size([64])

Epoch 5/10, Loss: 0.249582932472229
logits:  torch.Size([64, 2]) label:  torch.Size([64])

Epoch 6/10, Loss: 0.17822025448083878
logits:  torch.Size([64, 2]) label:  torch.Size([64])

Epoch 7/10, Loss: 0.12991637319326402
logits:  torch.Size([64, 2]) label:  torch.Size([64])

Epoch 8/10, Loss: 0.09810113298892975
logits:  torch.Size([64, 2]) label:  torch.Size([64])

Epoch 9/10, Loss: 0.07552705666422845
logits:  torch.Size([64, 2]) label:  torch.Size([64])

Epoch 10/10, Loss: 0.060588793426752094
logits:  torch.Size([64, 2]) label:  torch.Size([64])



## from chat gpt

In the context of training a neural network with PyTorch, `optimizer.zero_grad()` plays a crucial role in the optimization process. Let's break down its purpose and how it fits into the training loop.

### Purpose of `optimizer.zero_grad()`

**1. Clearing Previous Gradients:**
   - During the backpropagation step (`loss.backward()`), PyTorch computes the gradients of the loss with respect to the model parameters and accumulates them in the `.grad` attributes of each parameter. If `optimizer.zero_grad()` is not called, these gradients will accumulate (i.e., they will be summed) across multiple batches. This is usually not desired because each backpropagation step should be independent and only based on the current batch of data.

**2. Ensuring Correct Gradient Updates:**
   - By zeroing the gradients at the start of each iteration, you ensure that each parameter update is based only on the gradients calculated from the current batch. This helps in ensuring that the model learns correctly and efficiently.


### Detailed Step-by-Step Explanation

1. **Step 1: Clear Previous Gradients**
   - `optimizer.zero_grad()`: This ensures that gradients from the previous batch are not accumulated. This clears the `.grad` attribute of each parameter before computing new gradients.

2. **Step 2: Forward Pass**
   - `logits = model(inputs)`: The model processes the input data and produces predictions (logits).

3. **Step 3: Compute Loss**
   - `loss = criterion(logits, labels)`: The loss function compares the model's predictions with the true labels to compute the loss.

4. **Step 4: Backward Pass**
   - `loss.backward()`: This computes the gradient of the loss with respect to each parameter of the model. These gradients are stored in the `.grad` attributes of the model's parameters.

5. **Step 5: Update Model Parameters**
   - `optimizer.step()`: This updates the model's parameters using the gradients computed in the backward pass. The optimizer performs a step of gradient descent (or another optimization algorithm).

6. **Step 6: Accumulate Loss for Reporting**
   - `running_loss += loss.item()`: This accumulates the loss for the current batch to compute the average loss over the epoch.

By zeroing the gradients at the beginning of each batch, you ensure that each parameter update is based solely on the gradients calculated from the current batch, leading to correct and stable training.

## Inference from Model

In [ ]:
sample = torch.from_numpy(all_pos_int_tweets_matrix[0,:]).long()
sample


tensor([3, 4, 5, 6, 7, 8, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
sample.shape

torch.Size([60])

In [ ]:
sample = sample.unsqueeze(0)
sample.shape

torch.Size([1, 60])

In [ ]:
model.eval()

SentimentAnalysisModel(
  (embedding): Embedding(9088, 256)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)

In [ ]:
# with torch.no_grad():
#     outputs = model(sample)
#     predictions = torch.sigmoid(outputs).squeeze()

# predictions


In [ ]:
sentiment = model(sample)
sentiment


tensor([[-2.2480,  2.7726]], grad_fn=<AddmmBackward0>)

In [ ]:
sentiment[0]

tensor([-2.2480,  2.7726], grad_fn=<SelectBackward0>)

In [ ]:
sentiment[0].shape

torch.Size([2])

In [ ]:
sentiment = sentiment[0].unsqueeze(0)
sentiment

tensor([[-2.2480,  2.7726]], grad_fn=<UnsqueezeBackward0>)

In [ ]:
sentiment.shape

torch.Size([1, 2])

In [ ]:
sentiment[0]

tensor([-2.2480,  2.7726], grad_fn=<SelectBackward0>)

In [ ]:
torch.softmax(sentiment[0], dim=0)

tensor([0.0066, 0.9934], grad_fn=<SoftmaxBackward0>)

In [ ]:
torch.log_softmax(sentiment[0], dim=0)


tensor([-5.0271, -0.0066], grad_fn=<LogSoftmaxBackward0>)

### Checking the model layer by layer

## Acessing the layers of the model i.e. model's attributes

In [ ]:
# # Step 2: Define the neural network architecture
# class SentimentAnalysisModel(nn.Module):
#     def __init__(self, vocab_size, embed_dim):
#         super(SentimentAnalysisModel, self).__init__()

#         self.embedding  =  nn.Embedding(vocab_size, embed_dim)
#         self.fc         =  nn.Linear(embed_dim, 2)  # Output dimension is 2 for softmax

#         # self.softmax = nn.Softmax(dim=1)   # Softmax over the classes


#     def forward(self, x):
#         embedded  =  self.embedding(x)
#         embedded  =  embedded.mean(dim=1)  # Average embeddings over the sequence length
#         logits    =  self.fc(embedded)
#         return logits  # Return logits for CrossEntropyLoss


In [ ]:
emb_out = model.embedding(torch.tensor(all_pos_int_tweets_matrix[0:5, :]).long())
emb_out


tensor([[[-1.0469,  0.1570, -0.8016,  ..., -0.7602, -0.8726,  0.8905],
         [-0.8407,  0.8203, -1.4808,  ...,  1.2311, -0.4834, -1.1950],
         [-0.6799,  1.1161, -0.7586,  ..., -1.9379,  0.4538,  0.9793],
         ...,
         [ 1.3383, -0.8536,  1.0025,  ..., -0.2773,  0.1708,  0.1508],
         [ 1.3383, -0.8536,  1.0025,  ..., -0.2773,  0.1708,  0.1508],
         [ 1.3383, -0.8536,  1.0025,  ..., -0.2773,  0.1708,  0.1508]],

        [[ 1.1541, -1.1380,  0.0074,  ...,  1.1503,  0.3124, -0.6342],
         [-0.1839, -0.0201,  0.5426,  ..., -0.9350,  0.4004,  0.0916],
         [-2.2354, -0.8770, -0.0563,  ..., -0.6899, -0.1936,  0.0255],
         ...,
         [ 1.3383, -0.8536,  1.0025,  ..., -0.2773,  0.1708,  0.1508],
         [ 1.3383, -0.8536,  1.0025,  ..., -0.2773,  0.1708,  0.1508],
         [ 1.3383, -0.8536,  1.0025,  ..., -0.2773,  0.1708,  0.1508]],

        [[ 2.2128,  0.6216, -1.3216,  ...,  1.2033,  0.1313, -0.8164],
         [-0.4332, -1.8818, -0.5156,  ..., -0

In [ ]:
emb_out.shape

torch.Size([5, 60, 256])

In [ ]:
emb_out_mean = emb_out.mean(dim=1)
emb_out_mean

tensor([[ 1.2094, -0.7216,  0.8588,  ..., -0.2970,  0.2088,  0.1938],
        [ 0.9746, -0.6268,  0.7783,  ..., -0.1478,  0.2146,  0.1289],
        [ 1.1735, -0.7113,  0.8628,  ..., -0.2236,  0.2682,  0.1801],
        [ 1.3039, -0.8186,  0.9810,  ..., -0.3055,  0.1751,  0.1961],
        [ 1.0031, -0.6838,  0.7074,  ..., -0.3293,  0.1914,  0.2926]],
       grad_fn=<MeanBackward1>)

In [ ]:
emb_out_mean.shape


torch.Size([5, 256])

In [ ]:
fc_out = model.fc(emb_out_mean)
fc_out


tensor([[-2.2480,  2.7726],
        [-1.6415,  2.1199],
        [-1.8441,  2.3707],
        [-1.4450,  2.0122],
        [-1.5970,  2.0830]], grad_fn=<AddmmBackward0>)

In [ ]:
# direct output
model(torch.tensor(all_pos_int_tweets_matrix[0:5, :]).long())

tensor([[-2.2480,  2.7726],
        [-1.6415,  2.1199],
        [-1.8441,  2.3707],
        [-1.4450,  2.0122],
        [-1.5970,  2.0830]], grad_fn=<AddmmBackward0>)

In [ ]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Create labels
labs = np.ones(5)  # Example labels, normally this would come from your data
labs = torch.from_numpy(labs).long()
print(labs.shape)  # torch.Size([5])

# Calculate the loss
loss_value = criterion(fc_out, labs)
print(loss_value)

torch.Size([5])
tensor(0.0200, grad_fn=<NllLossBackward0>)


# Doing same but by creating new individual layers

In [ ]:
# # Step 2: Define the neural network architecture
# class SentimentAnalysisModel(nn.Module):
#     def __init__(self, vocab_size, embed_dim):
#         super(SentimentAnalysisModel, self).__init__()

#         self.embedding  =  nn.Embedding(vocab_size, embed_dim)
#         self.fc         =  nn.Linear(embed_dim, 2)  # Output dimension is 2 for softmax

#         # self.softmax = nn.Softmax(dim=1)   # Softmax over the classes


#     def forward(self, x):
#         embedded  =  self.embedding(x)
#         embedded  =  embedded.mean(dim=1)  # Average embeddings over the sequence length
#         logits    =  self.fc(embedded)
#         return logits  # Return logits for CrossEntropyLoss


In [ ]:
vocab_size = len(Vocab)
embed_dim  = 256

embedding_l = nn.Embedding(vocab_size, embed_dim)


In [ ]:
emb_out = embedding_l(torch.tensor(all_pos_int_tweets_matrix[0:5, :]).long())
emb_out

tensor([[[-0.6163, -0.2989,  1.0442,  ...,  1.2361,  1.7564, -1.3446],
         [ 0.6693,  0.8019, -1.3153,  ...,  1.1763, -0.5617, -2.2972],
         [ 0.6873, -0.9628,  0.9584,  ..., -0.7802,  1.6621,  1.0394],
         ...,
         [-0.7801, -0.8463, -0.5342,  ...,  0.9044, -0.0411,  1.6866],
         [-0.7801, -0.8463, -0.5342,  ...,  0.9044, -0.0411,  1.6866],
         [-0.7801, -0.8463, -0.5342,  ...,  0.9044, -0.0411,  1.6866]],

        [[-1.0554, -0.1916,  1.1963,  ...,  1.5496,  0.7966,  2.0364],
         [-1.6666, -0.3436, -1.0833,  ..., -1.2104, -0.6063,  0.3768],
         [-0.0413, -0.5167,  1.6903,  ..., -0.4442,  0.2959,  1.0117],
         ...,
         [-0.7801, -0.8463, -0.5342,  ...,  0.9044, -0.0411,  1.6866],
         [-0.7801, -0.8463, -0.5342,  ...,  0.9044, -0.0411,  1.6866],
         [-0.7801, -0.8463, -0.5342,  ...,  0.9044, -0.0411,  1.6866]],

        [[-0.7558,  0.4669, -0.1737,  ..., -2.4953, -0.1961,  2.0060],
         [-0.8040,  0.1721,  1.7595,  ..., -0

In [ ]:
emb_out.shape

torch.Size([5, 60, 256])

In [ ]:
emb_out_mean = emb_out.mean(dim=1)
emb_out_mean

tensor([[-0.7306, -0.6844, -0.4117,  ...,  0.8305,  0.0019,  1.3776],
        [-0.7020, -0.6673, -0.4447,  ...,  0.6698,  0.0739,  1.3668],
        [-0.6642, -0.6486, -0.4283,  ...,  0.7389, -0.0488,  1.3870],
        [-0.7676, -0.7977, -0.4845,  ...,  0.8950, -0.0342,  1.6236],
        [-0.4928, -0.5768, -0.3473,  ...,  0.6053, -0.0243,  1.2973]],
       grad_fn=<MeanBackward1>)

In [ ]:
emb_out_mean.shape

torch.Size([5, 256])

In [ ]:
fc = nn.Linear(embed_dim, 2)


In [ ]:
fc_out = fc(emb_out_mean)
fc_out

tensor([[0.0128, 0.3910],
        [0.0232, 0.3510],
        [0.0544, 0.3874],
        [0.0644, 0.4575],
        [0.0208, 0.2836]], grad_fn=<AddmmBackward0>)

In [ ]:
soft_l = nn.Softmax(dim=1)

In [ ]:
soft_l(fc_out)


tensor([[0.4066, 0.5934],
        [0.4188, 0.5812],
        [0.4175, 0.5825],
        [0.4030, 0.5970],
        [0.4347, 0.5653]], grad_fn=<SoftmaxBackward0>)

In [ ]:
fc_out.shape

torch.Size([5, 2])

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# # Dummy example for the output of the fully connected layer
# fc_out = torch.randn(5, 2)  # Example output, normally this would come from your model
# print(fc_out.shape)  # torch.Size([5, 2])

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Create labels
labs = np.ones(5)  # Example labels, normally this would come from your data
labs = torch.from_numpy(labs).long()
print(labs.shape)  # torch.Size([5])

# Calculate the loss
loss_value = criterion(fc_out, labs)
print(loss_value)


torch.Size([5])
tensor(0.5382, grad_fn=<NllLossBackward0>)


# Using the last variation with train, validation and test datasets

In [ ]:
# validation data

# Convert to PyTorch tensors
val_x_torch = torch.tensor(val_X, dtype=torch.long)
val_y_torch = torch.tensor(val_Y, dtype=torch.long)  # .long for CrossEntropyLoss

# Create a TensorDataset and DataLoader
val_dataset  =  TensorDataset(val_x_torch, val_y_torch)
val_gen      =  DataLoader(val_dataset,    batch_size=64, shuffle=True)


In [ ]:
# train data

# Convert to PyTorch tensors
train_x_torch = torch.tensor(train_X, dtype=torch.long)
train_y_torch = torch.tensor(train_Y, dtype=torch.long)  # .long for CrossEntropyLoss

# Create a TensorDataset and DataLoader
train_dataset  =  TensorDataset(train_x_torch, train_y_torch)
train_gen      =  DataLoader(train_dataset,    batch_size=64, shuffle=True)


In [ ]:
len(list(train_dataset))

8000

In [ ]:
dataset

In [ ]:
list(train_dataset)[7000]

(tensor([8339, 3760,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0]),
 tensor(0))

In [ ]:
list(train_dataset)[1000]


(tensor([   7, 2100,    9,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0]),
 tensor(1))

In [ ]:
type(list(dataset)[1000])

tuple

In [ ]:
dataloader


In [ ]:
list(train_gen)


[[tensor([[  95,   22,   14,  ...,    0,    0,    0],
          [ 317,   44,  318,  ...,    0,    0,    0],
          [1001,    9,    0,  ...,    0,    0,    0],
          ...,
          [  22, 7876, 7877,  ...,    0,    0,    0],
          [ 705,  308,   52,  ...,    0,    0,    0],
          [2895, 3760,    0,  ...,    0,    0,    0]]),
  tensor([0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
          1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
          0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])],
 [tensor([[  24,  327, 1964,  ...,    0,    0,    0],
          [  95, 1871,    0,  ...,    0,    0,    0],
          [ 144,  376,  294,  ...,    0,    0,    0],
          ...,
          [3760,  416,   53,  ...,    0,    0,    0],
          [ 344,  272,  336,  ...,    0,    0,    0],
          [  56,  375,    9,  ...,    0,    0,    0]]),
  tensor([1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
          0

In [ ]:
len(list(train_gen)) # 125 batch pray huye hain


125

In [ ]:
list(train_gen)[0]  # first batch

[tensor([[ 325, 4509,    9,  ...,    0,    0,    0],
         [ 601, 3283,  510,  ...,    0,    0,    0],
         [ 458,  229, 4245,  ...,    0,    0,    0],
         ...,
         [ 808, 5782, 5747,  ...,    0,    0,    0],
         [1961, 1106,   92,  ...,    0,    0,    0],
         [3225,  744, 3760,  ...,    0,    0,    0]]),
 tensor([1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
         1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])]

In [ ]:
type(list(train_gen)[0])


list

In [ ]:
len(list(train_gen)[0])  # 0: tensor tweets, 1: corresponding labels 0s and 1s


2

In [ ]:
list(train_gen)[0][0]  # har new call par kuch oor hi aaye ga


tensor([[ 349,   75,  253,  ...,    0,    0,    0],
        [2340, 2685,   75,  ...,    0,    0,    0],
        [1528,  579,  179,  ...,    0,    0,    0],
        ...,
        [ 427, 1027, 3948,  ...,    0,    0,    0],
        [6938,  308,  394,  ...,    0,    0,    0],
        [ 131,  242, 1017,  ...,    0,    0,    0]])

In [ ]:
len(list(train_gen)[0][0])


64

In [ ]:
list(train_gen)[0][1]

tensor([0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
        0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1])

In [ ]:
list(train_gen)[0][1] == 0

tensor([ True,  True, False,  True, False, False, False, False, False, False,
         True,  True, False,  True, False,  True, False, False,  True,  True,
        False,  True, False, False,  True, False,  True, False,  True, False,
         True,  True, False,  True,  True, False, False,  True, False, False,
         True,  True,  True, False,  True, False, False,  True,  True,  True,
         True,  True, False, False,  True, False,  True, False, False, False,
         True, False,  True,  True])

In [ ]:
torch.sum(list(train_gen)[0][1] == 0)
# yani 64 main say 38 0s hain

tensor(32)

### Model Architecture

In [ ]:
# Step 2: Define the neural network architecture
class SentimentAnalysisModel(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(SentimentAnalysisModel, self).__init__()

        self.embedding  =  nn.Embedding(vocab_size, embed_dim)
        self.fc         =  nn.Linear(embed_dim, 2)  # Output dimension is 2 for softmax

        # self.softmax = nn.Softmax(dim=1)   # Softmax over the classes


    def forward(self, x):
        embedded  =  self.embedding(x)
        embedded  =  embedded.mean(dim=1)  # Average embeddings over the sequence length
        logits    =  self.fc(embedded)
        return logits  # Return logits for CrossEntropyLoss


In [ ]:
# Parameters
vocab_size = len(Vocab)
embed_dim  = 256

# Instantiate the model
model = SentimentAnalysisModel(vocab_size, embed_dim)


In [ ]:
# Step 3: Define loss function and optimizer
criterion  =  nn.CrossEntropyLoss()  # Use CrossEntropyLoss for multi-class classification
optimizer  =  optim.Adam(model.parameters(), lr=0.001)


In [ ]:
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct      = (predicted == labels).sum().item()
    total        = labels.size(0)
    return correct / total


In [ ]:
# !pip install tensorboard


In [ ]:
# from torch.utils.tensorboard import SummaryWriter

# # Create a SummaryWriter instance
# writer = SummaryWriter(log_dir='runs/experiment_name')


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss  = 0.0
    correct_train = 0
    total_train   = 0

    for i, (inputs, labels) in enumerate(train_gen):
        optimizer.zero_grad()
        logits = model(inputs)
        loss   = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted   = torch.max(logits, 1)
        correct_train += (predicted == labels).sum().item()
        total_train   += labels.size(0)

    # Average training loss and accuracy
    avg_train_loss = running_loss / len(train_gen)
    train_accuracy = correct_train / total_train

    # # Log training metrics
    # writer.add_scalar('Training Loss', avg_train_loss, epoch)
    # writer.add_scalar('Training Accuracy', train_accuracy, epoch)

    # Validation
    model.eval()
    running_val_loss = 0.0
    correct_val      = 0
    total_val        = 0

    with torch.no_grad():
        for inputs, labels in val_gen:
            logits            = model(inputs)
            loss              = criterion(logits, labels)
            running_val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted  = torch.max(logits, 1)
            correct_val  += (predicted == labels).sum().item()
            total_val    += labels.size(0)

    # Average validation loss and accuracy
    avg_val_loss = running_val_loss / len(val_gen)
    val_accuracy = correct_val / total_val

    # # Log validation metrics
    # writer.add_scalar('Validation Loss', avg_val_loss, epoch)
    # writer.add_scalar('Validation Accuracy', val_accuracy, epoch)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    print()


Epoch 1/10
Training Loss: 0.6453, Training Accuracy: 0.7135
Validation Loss: 0.5302, Validation Accuracy: 0.9590

Epoch 2/10
Training Loss: 0.5250, Training Accuracy: 0.9062
Validation Loss: 0.3606, Validation Accuracy: 0.9675

Epoch 3/10
Training Loss: 0.3865, Training Accuracy: 0.9699
Validation Loss: 0.2120, Validation Accuracy: 0.9880

Epoch 4/10
Training Loss: 0.2683, Training Accuracy: 0.9888
Validation Loss: 0.1255, Validation Accuracy: 0.9900

Epoch 5/10
Training Loss: 0.1859, Training Accuracy: 0.9935
Validation Loss: 0.0813, Validation Accuracy: 0.9900

Epoch 6/10
Training Loss: 0.1332, Training Accuracy: 0.9940
Validation Loss: 0.0521, Validation Accuracy: 0.9940

Epoch 7/10
Training Loss: 0.0970, Training Accuracy: 0.9960
Validation Loss: 0.0403, Validation Accuracy: 0.9935

Epoch 8/10
Training Loss: 0.0735, Training Accuracy: 0.9964
Validation Loss: 0.0277, Validation Accuracy: 0.9955

Epoch 9/10
Training Loss: 0.0576, Training Accuracy: 0.9966
Validation Loss: 0.0223, Val

# [A discussion with chatgpt on call backs in pytorch](https://chatgpt.com/share/1a49c819-c3a2-403d-9cf6-9fd6418d8afe)

In [ ]:
# !tensorboard --logdir=runs


2024-07-22 12:38:29.868550: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 12:38:29.868627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 12:38:29.872223: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-22 12:38:31.667726: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to 

### from chat gpt
The `with torch.no_grad():` context manager in PyTorch is used to temporarily disable gradient calculation, which is crucial during the evaluation phase of model training. Here’s a detailed explanation of the motivation and benefits behind using `torch.no_grad()`:

### Motivation and Benefits

1. **Memory Efficiency:**
   - **Gradient Storage:** During training, PyTorch needs to store intermediate values and gradients for backpropagation. This requires additional memory. When you are only performing forward passes, such as during evaluation or inference, storing gradients is unnecessary and wasteful. By using `torch.no_grad()`, you avoid allocating memory for gradients, which can significantly reduce memory usage, especially with large models or large batch sizes.

2. **Computational Efficiency:**
   - **Unnecessary Computation:** Computing gradients involves additional operations that are not needed during evaluation. Disabling gradient computation reduces the amount of computation performed, speeding up the forward pass and making the evaluation phase faster.

3. **Avoiding Gradient Accumulation:**
   - **Gradient Accumulation Issue:** PyTorch accumulates gradients by default, which means gradients from previous batches are added to the current ones. This is intended for training but can lead to unintended behavior during evaluation if not disabled. Using `torch.no_grad()` ensures that no gradients are calculated or accumulated, which is what you want during validation or inference.

4. **Consistency with Training Mode:**
   - **Training vs. Evaluation:** When using `torch.no_grad()`, you clearly separate the phases where gradients are required (training) from those where they are not (evaluation). This helps in maintaining consistency and clarity in the code.

### Example Use Cases

1. **During Validation:**
   Validation is typically performed to evaluate the performance of the model on unseen data without updating model parameters. Here's how you use `torch.no_grad()` during validation:

2. **During Inference:**
   When deploying a model for inference or making predictions, you only need the forward pass to obtain predictions. Use `torch.no_grad()` to ensure that the model runs efficiently:

   ```python
   model.eval()  # Set model to evaluation mode
   with torch.no_grad():  # Disable gradient calculation
       predictions = model(input_data)
   ```

### Summary

- **Memory Efficiency:** Reduces memory usage by not storing gradients.
- **Computational Efficiency:** Speeds up the forward pass by avoiding unnecessary computations.
- **Avoids Unintended Gradient Accumulation:** Ensures no gradients are computed or stored.
- **Consistency:** Separates training and evaluation phases clearly.

Using `torch.no_grad()` is a best practice in PyTorch for evaluation and inference to ensure that your model runs efficiently and effectively during these phases.

# Testing on my data

In [ ]:
mytweet1 = ['my name is ans imran and i am a very very bad guy. go to hell']
mytweet2 = ['my name is ans imran and i am a very very good guy']
mytweet3 = ['i am nether a good guy, nor a bad guy, i am neutral']


my_tweets = mytweet1+mytweet2+mytweet3
my_labels = [0,1,0]

my_tweets, my_labels


(['my name is ans imran and i am a very very bad guy. go to hell',
  'my name is ans imran and i am a very very good guy',
  'i am nether a good guy, nor a bad guy, i am neutral'],
 [0, 1, 0])

In [ ]:
test_X, test_Y = list_of_sentences_to_numpy_sequences(my_tweets,Vocab, my_labels)

test_X = np.array(test_X)
test_Y = np.array(test_Y)

test_X = torch.from_numpy(test_X).long()
test_Y = torch.from_numpy(test_Y).long()


In [ ]:
type(test_X)


torch.Tensor

In [ ]:
model.eval()

SentimentAnalysisModel(
  (embedding): Embedding(9088, 256)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)

In [ ]:
with torch.no_grad():
  logits            = model(test_X)
  loss              = criterion(logits, test_Y)
  running_test_loss = loss.item()

  _, predicted  = torch.max(logits, 1)
    # correct_val  += (predicted == labels).sum().item()
    # total_val    += labels.size(0)
predicted, running_test_loss

(tensor([0, 1, 1]), 0.5841279029846191)